<a href="https://colab.research.google.com/github/satyakisen/pandas-ff-comparison/blob/main/Pandas_File_Format_Comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pandas File Format Comparision
## Overview
In this notebook let us compare the following pandas file formats.
1. csv - common text file that is comma seperated.
2. hdf5 - an open source file format that supports large, complex, heterogeneous data
3. parquet - an open source, column-oriented data file format designed for efficient data storage and retrieval.
4. feather - a portable file format for storing Arrow tables or data frames (from languages like Python or R) that utilizes the Arrow IPC format internally

Comparision parameters to consider are:
1. Time to write.
2. Time to read.
3. File size on disk.

## Let us begin
Let us begin by creating a dummy dataset containing only random float.

In [ ]:
import pandas as pd
import numpy as np

def make_data(row_n, col_n):
  arr = np.random.randn(row_n, col_n)
  df = pd.DataFrame(arr, columns=['col_{0}'.format(i) for i in range(col_n)])
  return df

df = make_data(100000, 10)

Let us check the dummy dataset we made.

In [ ]:
df.head(5)

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9
0,0.565050,-0.464023,-0.959550,-1.967708,-0.452082,0.630910,0.689905,-1.107446,0.445013,1.512598
1,-1.987039,1.880657,0.710337,-0.461089,0.327955,0.584324,1.118291,-1.341915,-0.689501,0.745430
2,-0.146080,-0.189735,-2.051159,-1.078399,-0.310501,0.409126,0.531062,0.533826,1.951706,0.742404
3,-0.322481,0.633855,0.111422,-0.759277,-0.237721,-1.336534,0.272647,-0.003573,1.703583,-0.444077
4,-1.019984,-1.109473,-0.476630,0.394867,1.024411,0.443024,1.286248,-0.202661,0.605197,0.143585


## Time to write

Let us now check the writing time of the above dataframe we created. We will first create a decorator for calculating time and then write a function to save the dataframe into different file formats.

In [23]:
%time df.to_csv('test.csv')

CPU times: user 784 ms, sys: 31.4 ms, total: 815 ms
Wall time: 818 ms
